In [6]:
import boto3
import os
import tarfile
import joblib
import xgboost as xgb
import json
import time
from dotenv import load_dotenv
from sagemaker import image_uris

C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/28/25 09:27:47] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=154563;file://C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=293547;file://C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Himanshu\AppData\Local\sagemaker\sagemaker\config.yaml


In [7]:


# ✅ Step 1: Load Environment Variables (Secrets)
load_dotenv()  # Load from .env file

BUCKET_NAME = os.getenv("SAGEMAKER_BUCKET")
ROLE_ARN = os.getenv("SAGEMAKER_ROLE")

if not BUCKET_NAME or not ROLE_ARN:
    raise ValueError("❌ Missing AWS credentials in .env file!")

In [2]:
pip install load_dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:


# ✅ Step 2: Define Model & Endpoint Names
MODEL_NAME = "xgboost-demand-forecasting"
ENDPOINT_CONFIG_NAME = MODEL_NAME + "-config"
ENDPOINT_NAME = MODEL_NAME + "-endpoint"
MODEL_FILE = "xgboost_model.tar.gz"
MODEL_URI = f"s3://{BUCKET_NAME}/{MODEL_FILE}"

In [9]:


# ✅ Step 3: Get the Correct XGBoost Image URI
region = boto3.Session().region_name
xgboost_image_uri = image_uris.retrieve("xgboost", region, version="1.5-1")

[02/28/25 09:27:50] INFO     Ignoring unnecessary instance type: None.                            ]8;id=198242;file://C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=604463;file://C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\sagemaker\image_uris.py#530\530]8;;\

In [10]:
# ✅ Step 4: Initialize SageMaker & S3 Clients
sagemaker_client = boto3.client("sagemaker")
s3 = boto3.client("s3")

[02/28/25 09:27:53] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=287651;file://C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=608624;file://C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

In [11]:
def delete_if_exists():
    # ✅ Delete existing SageMaker endpoint
    try:
        sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)
        sagemaker_client.delete_endpoint(EndpointName=ENDPOINT_NAME)
        print(f"✅ Deleted existing endpoint: {ENDPOINT_NAME}")
    except sagemaker_client.exceptions.ClientError:
        print(f"✅ No existing endpoint found.")

    # ✅ Delete existing SageMaker endpoint configuration
    try:
        sagemaker_client.describe_endpoint_config(EndpointConfigName=ENDPOINT_CONFIG_NAME)
        sagemaker_client.delete_endpoint_config(EndpointConfigName=ENDPOINT_CONFIG_NAME)
        print(f"✅ Deleted existing endpoint config: {ENDPOINT_CONFIG_NAME}")
    except sagemaker_client.exceptions.ClientError:
        print(f"✅ No existing endpoint config found.")

    # ✅ Delete existing SageMaker model
    try:
        sagemaker_client.describe_model(ModelName=MODEL_NAME)
        sagemaker_client.delete_model(ModelName=MODEL_NAME)
        print(f"✅ Deleted existing model: {MODEL_NAME}")
    except sagemaker_client.exceptions.ClientError:
        print(f"✅ No existing model found.")

# Run the cleanup function before deploying the new model
delete_if_exists()


✅ No existing endpoint found.
✅ No existing endpoint config found.
✅ No existing model found.


In [12]:

# ✅ Step 6: Convert .pkl Model → XGBoost Native .model & Package
MODEL_DIR = "../models/"
os.makedirs(MODEL_DIR, exist_ok=True)

In [13]:
import os

MODEL_DIR = "../models/"
pkl_model_path = os.path.join(MODEL_DIR, "xgboost_model.pkl")

# Check if the file exists
if os.path.exists(pkl_model_path):
    print(f"✅ Found model: {pkl_model_path}")
else:
    print(f"❌ Model not found at {pkl_model_path}. Did you save it correctly in `04_model_training.ipynb`?")


✅ Found model: ../models/xgboost_model.pkl


In [14]:

# Load trained model from .pkl
pkl_model_path = os.path.join(MODEL_DIR, "xgboost_model.pkl")
xgb_model = joblib.load(pkl_model_path)

In [15]:

# Save in XGBoost's native format
xgb_model_path = os.path.join(MODEL_DIR, "xgboost_model.model")
xgb_model.save_model(xgb_model_path)

C:\Users\Himanshu\Demand_Forecasting_Project\forecasting_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:28:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [16]:
# Apply correct file permissions before packaging
os.chmod(xgb_model_path, 0o644)  # Read & write for owner, read-only for others


In [17]:

# Create .tar.gz package for SageMaker
tar_path = os.path.join(MODEL_DIR, MODEL_FILE)
with tarfile.open(tar_path, "w:gz") as tar:
    tar.add(xgb_model_path, arcname="xgboost_model.model")

print(f"✅ Model converted, permissions fixed, and compressed at {tar_path}")

✅ Model converted, permissions fixed, and compressed at ../models/xgboost_model.tar.gz


In [18]:

# ✅ Step 7: Upload Model to S3
s3.upload_file(tar_path, BUCKET_NAME, MODEL_FILE)
print(f"✅ Model uploaded to S3: {MODEL_URI}")

✅ Model uploaded to S3: s3://demand-forecasting-bucket-us-east-1/xgboost_model.tar.gz


In [19]:



# ✅ Step 8: Create SageMaker Model
print("🚀 Creating new SageMaker model...")
sagemaker_client.create_model(
    ModelName=MODEL_NAME,
    PrimaryContainer={"Image": xgboost_image_uri, "ModelDataUrl": MODEL_URI},
    ExecutionRoleArn=ROLE_ARN
)
print(f"✅ SageMaker model '{MODEL_NAME}' created.")

🚀 Creating new SageMaker model...
✅ SageMaker model 'xgboost-demand-forecasting' created.


In [20]:

# ✅ Step 9: Create Endpoint Configuration
print("🚀 Creating new endpoint configuration...")
sagemaker_client.create_endpoint_config(
    EndpointConfigName=ENDPOINT_CONFIG_NAME,
    ProductionVariants=[{
        "VariantName": "AllTraffic",
        "ModelName": MODEL_NAME,
        "InstanceType": "ml.m5.large",
        "InitialInstanceCount": 1
    }]
)
print(f"✅ Endpoint config '{ENDPOINT_CONFIG_NAME}' created.")

🚀 Creating new endpoint configuration...
✅ Endpoint config 'xgboost-demand-forecasting-config' created.


In [21]:

# ✅ Step 10: Deploy SageMaker Endpoint
print("🚀 Deploying new endpoint...")
sagemaker_client.create_endpoint(
    EndpointName=ENDPOINT_NAME,
    EndpointConfigName=ENDPOINT_CONFIG_NAME
)
print(f"🚀 Deploying SageMaker endpoint '{ENDPOINT_NAME}'... (This may take a few minutes)")

🚀 Deploying new endpoint...
🚀 Deploying SageMaker endpoint 'xgboost-demand-forecasting-endpoint'... (This may take a few minutes)


In [ ]:




# ✅ Step 11: Wait for Deployment to Complete
while True:
    response = sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)
    status = response["EndpointStatus"]
    print(f"🔍 Current Status: {status}")

    if status == "InService":
        print(f"✅ SageMaker Endpoint '{ENDPOINT_NAME}' is ready for inference!")
        break
    elif status in ["Failed", "RollingBack"]:
        print(f"❌ Deployment failed! Check AWS SageMaker logs.")
        
        # Get failure reason from SageMaker
        failure_reason = response.get("FailureReason", "Unknown error")
        print(f"🔴 Failure Reason: {failure_reason}")
        
        # Exit script
        raise SystemExit(f"❌ Deployment failed: {failure_reason}")
    
    time.sleep(30)  # Check status every 30 seconds

print("🎯 Deployment process completed successfully!")

🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating
🔍 Current Status: Creating


In [ ]:







# ✅ Step 12: Test Model Inference
print("🚀 Testing inference on deployed model...")

runtime = boto3.client("sagemaker-runtime")

# Create a sample input (Modify based on your data)
test_input = json.dumps({"instances": [[5.2, 3.1, 1.4, 0.2]]})

# Invoke the endpoint
response = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=test_input
)

# Parse response
result = json.loads(response["Body"].read().decode())
print("✅ Model Inference Output:", result)
